In [ ]:
%%writefile learners_file.py

import adaptive

import phase_diagram
lead_pars = dict(
    a=10, r1=50, r2=70, coverage_angle=135, angle=45, with_shell=True, which_lead=""
)

params = dict(
    alpha=20,
    B_x=0,
    B_y=0,
    B_z=0,
    Delta=110,
    g=50,
    orbital=True,
    mu_sc=100,
    c_tunnel=3 / 4,
    V_r=-50,
    intrinsic_sc=False,
    mu_=lambda x0, sigma, mu_lead, mu_wire: mu_lead,
    V_=lambda z, V_0, V_r, V_l, x0, sigma, r1: 0,
    V_0=None,
    V_l=None,
    mu_lead=10,
    mu_wire=None,
    r1=None,
    sigma=None,
    x0=None,
    **phase_diagram.constants.__dict__
)


def pf(xy, params=params, lead_pars=lead_pars):
    import phase_diagram 

    params["B_x"], params["mu_lead"] = xy
    lead = phase_diagram.make_lead(**lead_pars).finalized()
    return phase_diagram.calculate_pfaffian(lead, params)


def smallest_gap(xy, params=params, lead_pars=lead_pars):
    import phase_diagram

    params["B_x"], params["mu_lead"] = xy
    lead = phase_diagram.make_lead(**lead_pars).finalized()
    pf = phase_diagram.calculate_pfaffian(lead, params)
    gap = phase_diagram.gap_from_modes(lead, params)
    return pf * gap

fname = 'phase_diagram_gap.pickle'
# fname = 'phase_diagram.pickle'

loss = adaptive.learner.learnerND.curvature_loss_function()
learner = adaptive.LearnerND(smallest_gap, bounds=[(0, 2), (0, 35)], loss_per_simplex=loss)

# learner.load(fname)

learners = [learner]
fnames = [fname]

In [ ]:
import adaptive
adaptive.notebook_extension()
runner = adaptive.Runner(learner, goal=lambda l: l.npoints > 60000)
runner.live_info()

In [ ]:
%%output size=100
learner.plot(tri_alpha=0.4, n=None)

In [ ]:
learner.save(fname)

In [ ]:
import adaptive_scheduler

def goal(learner):
    return learner.npoints > 200

scheduler = adaptive_scheduler.scheduler.DefaultScheduler(
    cores=100,
    executor_type="ipyparallel",
)  # PBS or SLURM

run_manager = adaptive_scheduler.server_support.RunManager(
    scheduler=scheduler,
    learners_file="learners_file.py",
    goal=goal,
    log_interval=30,
    save_interval=30,
    job_name='phase-diagram'
)
run_manager.start()